In [1]:
import numpy as np
from scipy import signal
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.preprocessing import LabelBinarizer
# from mne import Epochs, pick_types, find_events
# from mne.channels import read_layout
# from mne.io import concatenate_raws, read_raw_edf
# from mne.datasets import eegbci
# from mne.decoding import CSP
import csv
import pickle
import datetime


In [2]:
# set path to datasets
savetag='_exp7'

datapath = '/home/jingyan/Documents/ME499-WinterProject/mindwave/src/motor_cortex_ml/data/'
csvname =datapath + 'record'+savetag+'.csv'
# csvname =datapath +'record_20rfft.csv'
data = np.loadtxt(csvname,delimiter = ',')
savedsp=datapath+'dsp'+savetag+'.csv'
# get input/output pairs
x = data[:,:-1]
y = data[:,-1]
channel_num=8
chunk=40 #matches self.recordsize in mindwave_record.py
fs=250
band=(7,13)

def bandpass(start,stop,data,fs):
    bp_Hz = np.array([start, stop])
    b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
    return signal.lfilter(b, a, data, axis=0)

def notch(val, data, fs):
    bp_stop_Hz = val + 3.0 * np.array([-1, 1])
    b, a = signal.butter(3, bp_stop_Hz / (fs / 2.0), 'bandstop')
    return signal.lfilter(b, a, data)

def fft(data, fs):
    L = len(data)
    freq = np.linspace(0.0, 1.0 / (2.0 * fs **-1), int(L / 2))
    yi = np.fft.fft(data)#[1:]
    y = yi[range(int(L / 2))]
    # ysample= (2.0*abs(yi/L))[range(int(L/2))]
    return freq, abs(y)


In [3]:
for k in range(channel_num):
    locals()["channel"+str(k+1)]=x[:,k]
#Divide datas to chunks, each chunk coorespond to specific channel with specific y value.
#Total chunk number = channel_num * record_size(chunk) *4
_total_chunk=channel_num*chunk*4
for i in np.unique(y):
    i=int(i)
    locals()['ind_'+str(i+1)]=np.where(y==i)[0] ##left=-1,(ind0) ; right=1,(ind2); focus=0,(ind1);
   
    locals()['ind'+str(i+1)]=[locals()['ind_'+str(int(i+1))][0]]
    for j in range(len(locals()['ind_'+str(i+1)])-1):  
        if (locals()['ind_'+str(i+1)])[j]+1!=(locals()['ind_'+str(i+1)])[j+1]:
            locals()['ind'+str(i+1)].append((locals()['ind_'+str(i+1)])[j])
        if j>1 and (locals()['ind_'+str(i+1)])[j]-1!=(locals()['ind_'+str(i+1)])[j-1]:
            (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[j])
    (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[-1])
    if i!=0 and 2*chunk!=len((locals()['ind'+str(i+1)])):
        print("error: chunk size and interval limit does not match. flag number:",i)
    elif i==0 and 4*chunk!=len((locals()['ind'+str(i+1)])):
        print("error: chunk size and interval limit does not match. flag number: ",i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]=locals()['channel'+str(k+1)][(locals()['ind'+str(i+1)])[2*g]:(locals()['ind'+str(i+1)])[2*g+1]+1]
        

In [7]:
#Run this cell if doing FFT
#DSP for individual chunk###########FFT
total_chunks=0
row=''
#generate average amplitude of each channel for indivisual label in specific band range
#variable_channel#_label#_chunk#

for i in np.unique(y):
    i=int(i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
            
################################dsp:fft##################################
            freq,ffty=fft(data,fs)
            freq_ind=np.where((freq>=band[0])&(freq<=band[1]))[0]
            desired_freq=freq[freq_ind]
            amp_of_desired_freq=ffty[freq_ind]
            average_amp=np.mean(amp_of_desired_freq)
            locals()['average_amp_'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=average_amp

            total_chunks+=1
if total_chunks != _total_chunk: print('error: total chunks number does not match with the data acqusition')

###################FFT input file generation##############################
for i in np.unique(y):
    i=int(i)
    for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            for k in range(channel_num):
                row+= str(locals()['average_amp_'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)])
                row+=','
            row+=str(i)
            row+='\n'
with open(savedsp,'w') as f:
    f.write(row)
dspdata = np.loadtxt(savedsp,delimiter = ',')
dspx = dspdata[:,:-1]
dspy = dspdata[:,-1]

In [30]:
#Run this cell if doing CSP
#DSP for individual chunk###################CSP
x_2class=np.append(x[ind_0],x[ind_2],axis=0)
y_2class=np.append(y[ind_0],y[ind_2],axis=0)
x=x_2class
y=y_2class
total_chunks=0
#generate average amplitude of each channel for indivisual label in specific band range
#variable_channel#_label#_chunk#

for i in np.unique(y):
    i=int(i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
            channel_filtered=bandpass(band[0],band[1],data,fs)
            locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=channel_filtered
            total_chunks+=1
if total_chunks != _total_chunk: print('error: total chunks number does not match with the data acqusition')

    
##################CSP input matrix generation#################################
labels=[]
for i in np.unique(y): 
    i=int(i)
    for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
        for k in range(channel_num):
            data=locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]
            if k==0:
                CSPmatrices_mergechn=[data]
            else:
                CSPmatrices_mergechn=np.append(CSPmatrices_mergechn,[data],axis=0)
        if g==0 and i==-1: CSPmatrices=CSPmatrices_mergechn.reshape(1,CSPmatrices_mergechn.shape[0],CSPmatrices_mergechn.shape[1])
        else: CSPmatrices=np.append(CSPmatrices,CSPmatrices_mergechn.reshape(1,CSPmatrices_mergechn.shape[0],CSPmatrices_mergechn.shape[1]),axis=0)
        if i!=1: l=1
        else: l=0
        labels=np.append(labels,[l]) #####label =l -> different moving vs not moving   ####if previous cell ran: label = i for left vs right movement


error: total chunks number does not match with the data acqusition


In [7]:
#Run this cell if doing CNN
#DSP for individual chunk###################CNN
total_chunks=0
#generate average amplitude of each channel for indivisual label in specific band range
#variable_channel#_label#_chunk#

for i in np.unique(y):
    i=int(i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
            channel_filtered=bandpass(band[0],band[1],data,fs)
            locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=channel_filtered
            total_chunks+=1
if total_chunks != _total_chunk: print('error: total chunks number does not match with the data acqusition')

    
##################CNN input matrix generation#################################
labels=[]
for i in np.unique(y): 
    i=int(i)
    for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
        for k in range(channel_num):
            data=locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]
            if k==0:
                CNNmatrices_mergechn=[data]
            else:
                CNNmatrices_mergechn=np.append(CNNmatrices_mergechn,[data],axis=0)
        if g==0 and i==-1: CNNmatrices=CNNmatrices_mergechn.reshape(1,CNNmatrices_mergechn.shape[0],CNNmatrices_mergechn.shape[1])
        else: CNNmatrices=np.append(CNNmatrices,CNNmatrices_mergechn.reshape(1,CNNmatrices_mergechn.shape[0],CNNmatrices_mergechn.shape[1]),axis=0)
        labels=np.append(labels,[i])
_shape=CNNmatrices.shape
CNN_input=CNNmatrices.reshape(_shape[0],1,_shape[1],_shape[2])
lb = LabelBinarizer()
label_enc=lb.fit_transform(labels)
np.savetxt("labels.csv", label_enc, delimiter=",")

In [8]:
label_enc.shape

(160, 3)

In [6]:
from EEGModels import EEGNet,ShallowConvNet,DeepConvNet
model1=EEGNet(nb_classes=3,Chans=8,Samples=1250)
model1.compile(loss='categorical_crossentropy',optimizer='adam')
model1.fit(CNN_input,label_enc,batch_size=5,epochs=20,verbose=2)

Epoch 1/20


InvalidArgumentError: Default MaxPoolingOp only supports NHWC on device type CPU
	 [[Node: max_pooling2d_2/MaxPool = MaxPool[T=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/max_pooling2d_2/MaxPool_grad/MaxPoolGrad"], data_format="NCHW", ksize=[1, 1, 1, 8], padding="VALID", strides=[1, 1, 1, 8], _device="/job:localhost/replica:0/task:0/device:CPU:0"](activation_2/Elu)]]

In [17]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 1, 8, 1250)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 40, 8, 1226)       1040      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 40, 1, 1226)       12800     
_________________________________________________________________
batch_normalization_6 (Batch (None, 40, 1, 1226)       160       
_________________________________________________________________
activation_18 (Activation)   (None, 40, 1, 1226)       0         
_________________________________________________________________
average_pooling2d_6 (Average (None, 40, 1, 77)         0         
_________________________________________________________________
activation_19 (Activation)   (None, 40, 1, 77)         0         
__________

In [25]:
csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
x_train=csp.fit_transform(CSPmatrices,labels)
lda =LinearDiscriminantAnalysis()
lda.fit(x_train,labels)
lda.score(x_train,labels)

Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.


0.75

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)


In [7]:
left_sp=3*x[:,2]-x[:,0]-x[:,4]-x[:,6]
right_sp=3*x[:,3]-x[:,1]-x[:,5]-x[:,7]
x_sp=np.append(left_sp.reshape(len(left_sp),1),right_sp.reshape(len(right_sp),1),axis=1)
x_train,x_test,y_train,y_test=train_test_split(x_sp,y,test_size=0.2)

In [12]:
model=KNeighborsClassifier(n_neighbors=5)
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

(0.84025, 0.74575)


In [33]:
model=GradientBoostingClassifier()
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

(0.6325625, 0.6055)


In [49]:
model=AdaBoostClassifier(n_estimators=60)#OneVsRestClassifier(SVC(kernel='linear'))
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

0.5040625 0.50825


In [86]:
model=OneVsRestClassifier(SVC(kernel='linear'))
model.fit(x,y)
score=model.score(x,y)
print(score)
model_filename='mindwave_model_exp4_ova.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

0.5438


In [27]:
model=SVC(gamma='scale', decision_function_shape='ovo')
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model_exp4_svc.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)


0.9696875 0.623


In [80]:
model=DecisionTreeClassifier()
model.fit(x,y)
score=model.score(x,y)
print(score)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

1.0


In [50]:
model=LinearDiscriminantAnalysis()
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

0.504375 0.5185
